# Visualize Token Probabilities and Model Confidence in LLM Predictions

In this tutorial we show how ``eli5.explain_prediction`` can be used to visualize LLM predictions,
highlighting tokens proportionally to the log probability. In many cases, this can help to see where
LLM is uncertain about its predictions:

![LLM token probabilities visualized with eli5.explain_prediction](../docs/source/static/llm-explain-logprobs.png)

## 1. OpenAI models

To follow this tutorial you need the ``openai`` library installed and working.

In [1]:
import eli5
import openai

client = openai.OpenAI()

First let's define our task: we'll be extracting specific product properties from a free-form product description:

In [2]:
product_description = """\
Stay is designed by Danish Maria Berntsen and is functional and beautiful lighting
in one and the same design. Stay has several nice features, where the lamp's flexible
swivel mechanism makes it possible to direct the light and create optimal lighting
conditions. Furthermore, the switch is discreetly placed behind the lamp head,
making it easy and convenient to turn the light on and off. Thus, Stay combines great
flexibility and elegant appearance in a highly convincing way.

Stay table lamp is highly functional, as the arm and head of the lamp can be adjusted
according to wishes and needs, which make it ideal as task lighting in the office.
Furthermore, the silky matte grey metal support the modern and minimalistic expression.\
"""
prompt = f"""\
Product description:
{product_description.strip()}

Extract the following properties from product description:
'materials' (list of strings),
'type' (string, e.g. 'LED'),
'color' (string),
'price' (non-zero float with 2 decimal places),
'summary' (string, a very short summary).
Respond with JSON dict with above keys. Always make sure to return the price or it's estimate.
"""
completion = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model="gpt-4o",
    logprobs=True,
    temperature=0,
)
print(completion.choices[0].message.content.strip('`'))

json
{
    "materials": ["metal"],
    "type": "task lighting",
    "color": "silky matte grey",
    "price": 150.00,
    "summary": "Stay table lamp with adjustable arm and head for optimal task lighting."
}



If you examine the prompt and description above, you can see that not all attributes are equally clear:
- "material" is quite clear and explicitly mentioned
- "color" is also clear, but its mention is more ambiguous
- "type" is underspecified, we don't extractly specify what we want,
- "summary" is quite clear, but there is no single correct summary,
- "price" is not present at all, but we ask a model to make a guess.

You may have noticed that above we included ``logprobs=True,`` in a call to the model,
this allows us to get a log-probability for each token, and then we can visualize them with ``eli5.explain_prediction``:

In [3]:
eli5.explain_prediction(completion)

Explanation(estimator='llm_logprobs', description=None, error=None, method=None, is_regression=False, targets=[TargetExplanation(target=Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='```', bytes=[96, 96, 96], logprob=-1.2233183042553719e-05, top_logprobs=[]), ChatCompletionTokenLogprob(token='json', bytes=[106, 115, 111, 110], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='\n', bytes=[10], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='{\n', bytes=[123, 10], logprob=-3.7697225252486533e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token='   ', bytes=[32, 32, 32], logprob=-0.31333035230636597, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='materials', bytes=[109, 97, 116, 101, 114, 105, 97, 108, 115], logprob=-1.9361264946837764e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token='":', bytes=[34, 58], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' ["', bytes=[32, 91, 34], logprob=-0.018151583150029182, top_logprobs=[]), ChatCompletionTokenLogprob(token='metal', bytes=[109, 101, 116, 97, 108], logprob=-0.001557355048134923, top_logprobs=[]), ChatCompletionTokenLogprob(token='"],\n', bytes=[34, 93, 44, 10], logprob=-0.0001307142956648022, top_logprobs=[]), ChatCompletionTokenLogprob(token='   ', bytes=[32, 32, 32], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='type', bytes=[116, 121, 112, 101], logprob=-1.9361264946837764e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token='":', bytes=[34, 58], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=-0.07268553972244263, top_logprobs=[]), ChatCompletionTokenLogprob(token='table', bytes=[116, 97, 98, 108, 101], logprob=-0.8554064631462097, top_logprobs=[]), ChatCompletionTokenLogprob(token=' lamp', bytes=[32, 108, 97, 109, 112], logprob=-0.00018125296628568321, top_logprobs=[]), ChatCompletionTokenLogprob(token='",\n', bytes=[34, 44, 10], logprob=-4.644463479053229e-05, top_logprobs=[]), ChatCompletionTokenLogprob(token='   ', bytes=[32, 32, 32], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='color', bytes=[99, 111, 108, 111, 114], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='":', bytes=[34, 58], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='sil', bytes=[115, 105, 108], logprob=-0.16051970422267914, top_logprobs=[]), ChatCompletionTokenLogprob(token='ky', bytes=[107, 121], logprob=-2.4391956685576588e-05, top_logprobs=[]), ChatCompletionTokenLogprob(token=' matte', bytes=[32, 109, 97, 116, 116, 101], logprob=-1.9361264946837764e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token=' grey', bytes=[32, 103, 114, 101, 121], logprob=-1.6240566083070007e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token='",\n', bytes=[34, 44, 10], logprob=-1.1472419600977446e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token='   ', bytes=[32, 32, 32], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='price', bytes=[112, 114, 105, 99, 101], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='":', bytes=[34, 58], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' ', bytes=[32], logprob=-1.3856492842023727e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token='150', bytes=[49, 53, 48], logprob=-1.2531815767288208, top_logprobs=[]), ChatCompletionTokenLogprob(token='.', bytes=[46], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='00', bytes=[48, 48], logprob=-9.019237768370

We can clearly see that the model is very confident in the material -- if you hover over the prediction, you can see a probability for each token -- and less confident about type and color. The confidence in price is a lot lower, while summary, being a longer piece of text, is harder to interpret -- we can see that some words follow more obviously.

We can also obtain the same result by passing ``client`` and ``prompt`` to ``eli5.explain_prediction``,
in this case it would call the client, and we can pass extra keyword arguments -- here we'll pass ``n=2`` to obtain two different predictions, and would leave temperature at default.

In [4]:
explanation = eli5.explain_prediction(client, prompt, model='gpt-4o', n=2)
explanation

Explanation(estimator='llm_logprobs', description=None, error=None, method=None, is_regression=False, targets=[TargetExplanation(target=Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='```', bytes=[96, 96, 96], logprob=-1.2233183042553719e-05, top_logprobs=[]), ChatCompletionTokenLogprob(token='json', bytes=[106, 115, 111, 110], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='\n', bytes=[10], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='{\n', bytes=[123, 10], logprob=-1.1472419600977446e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token='   ', bytes=[32, 32, 32], logprob=-0.1269671618938446, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='materials', bytes=[109, 97, 116, 101, 114, 105, 97, 108, 115], logprob=-1.9361264946837764e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token='":', bytes=[34, 58], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' ["', bytes=[32, 91, 34], logprob=-0.02975349873304367, top_logprobs=[]), ChatCompletionTokenLogprob(token='metal', bytes=[109, 101, 116, 97, 108], logprob=-0.0019255406223237514, top_logprobs=[]), ChatCompletionTokenLogprob(token='"],\n', bytes=[34, 93, 44, 10], logprob=-0.00012535012501757592, top_logprobs=[]), ChatCompletionTokenLogprob(token='   ', bytes=[32, 32, 32], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='type', bytes=[116, 121, 112, 101], logprob=-1.9361264946837764e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token='":', bytes=[34, 58], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "",\n', bytes=[32, 34, 34, 44, 10], logprob=-3.1719813346862793, top_logprobs=[]), ChatCompletionTokenLogprob(token='   ', bytes=[32, 32, 32], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='color', bytes=[99, 111, 108, 111, 114], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='":', bytes=[34, 58], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='sil', bytes=[115, 105, 108], logprob=-0.0868551954627037, top_logprobs=[]), ChatCompletionTokenLogprob(token='ky', bytes=[107, 121], logprob=-1.9027791495318525e-05, top_logprobs=[]), ChatCompletionTokenLogprob(token=' matte', bytes=[32, 109, 97, 116, 116, 101], logprob=-1.9361264946837764e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token=' grey', bytes=[32, 103, 114, 101, 121], logprob=-1.1472419600977446e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token='",\n', bytes=[34, 44, 10], logprob=-1.8624639324116288e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token='   ', bytes=[32, 32, 32], logprob=-1.9361264946837764e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='price', bytes=[112, 114, 105, 99, 101], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='":', bytes=[34, 58], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' ', bytes=[32], logprob=-2.0339031834737398e-05, top_logprobs=[]), ChatCompletionTokenLogprob(token='199', bytes=[49, 57, 57], logprob=-2.1854772567749023, top_logprobs=[]), ChatCompletionTokenLogprob(token='.', bytes=[46], logprob=-1.9361264946837764e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token='99', bytes=[57, 57], logprob=-0.021091928705573082, top_logprobs=[]), ChatCompletionTokenLogprob(token=',\n', bytes=[44, 10], logprob=-0.003600071184337139, top_logprobs=[]), ChatCompletionTokenLogprob(token='   ', bytes=[32, 32, 32], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=0.0, top_logprob

We can obtain the original prediction from the explanation object via ``explanation.targets[0].target``,
e.g. use ``explanation.targets[0].target.message.content`` to get the prediction text.

## 2. Limitations

Even though above the model confidence matched our expectations, it's not always the case.
For example, if we use "Chain of Thought" (https://arxiv.org/abs/2201.11903) reasoning,
asking the model first to think about the price estimate, it would be much more confident in the price in its final output,
but that does not reflect the real confidence of the model, as it's smeared over CoT:

In [6]:
prompt_cot = prompt + """
Before outputting the JSON with extracted results, provide analysis of the most likely price.
"""
eli5.explain_prediction(client, prompt_cot, model='gpt-4o', temperature=0)

Explanation(estimator='llm_logprobs', description=None, error=None, method=None, is_regression=False, targets=[TargetExplanation(target=Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='To', bytes=[84, 111], logprob=-0.4867655336856842, top_logprobs=[]), ChatCompletionTokenLogprob(token=' determine', bytes=[32, 100, 101, 116, 101, 114, 109, 105, 110, 101], logprob=-0.5725387930870056, top_logprobs=[]), ChatCompletionTokenLogprob(token=' the', bytes=[32, 116, 104, 101], logprob=-0.13688358664512634, top_logprobs=[]), ChatCompletionTokenLogprob(token=' most', bytes=[32, 109, 111, 115, 116], logprob=-0.19027188420295715, top_logprobs=[]), ChatCompletionTokenLogprob(token=' likely', bytes=[32, 108, 105, 107, 101, 108, 121], logprob=-0.0009278989746235311, top_logprobs=[]), ChatCompletionTokenLogprob(token=' price', bytes=[32, 112, 114, 105, 99, 101], logprob=-5.466968650580384e-05, top_logprobs=[]), ChatCompletionTokenLogprob(token=' for', bytes=[32, 102, 111, 114], logprob=-0.4798658788204193, top_logprobs=[]), ChatCompletionTokenLogprob(token=' the', bytes=[32, 116, 104, 101], logprob=-0.009040074422955513, top_logprobs=[]), ChatCompletionTokenLogprob(token=' Stay', bytes=[32, 83, 116, 97, 121], logprob=-0.2057265341281891, top_logprobs=[]), ChatCompletionTokenLogprob(token=' table', bytes=[32, 116, 97, 98, 108, 101], logprob=-0.004193079192191362, top_logprobs=[]), ChatCompletionTokenLogprob(token=' lamp', bytes=[32, 108, 97, 109, 112], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=',', bytes=[44], logprob=-0.137336865067482, top_logprobs=[]), ChatCompletionTokenLogprob(token=' we', bytes=[32, 119, 101], logprob=-0.36271974444389343, top_logprobs=[]), ChatCompletionTokenLogprob(token=' need', bytes=[32, 110, 101, 101, 100], logprob=-0.7786848545074463, top_logprobs=[]), ChatCompletionTokenLogprob(token=' to', bytes=[32, 116, 111], logprob=-0.0005338519695214927, top_logprobs=[]), ChatCompletionTokenLogprob(token=' consider', bytes=[32, 99, 111, 110, 115, 105, 100, 101, 114], logprob=-0.04549144208431244, top_logprobs=[]), ChatCompletionTokenLogprob(token=' several', bytes=[32, 115, 101, 118, 101, 114, 97, 108], logprob=-0.6360411047935486, top_logprobs=[]), ChatCompletionTokenLogprob(token=' factors', bytes=[32, 102, 97, 99, 116, 111, 114, 115], logprob=-0.037374746054410934, top_logprobs=[]), ChatCompletionTokenLogprob(token=':\n\n', bytes=[58, 10, 10], logprob=-1.1546510457992554, top_logprobs=[]), ChatCompletionTokenLogprob(token='1', bytes=[49], logprob=-0.00035929924342781305, top_logprobs=[]), ChatCompletionTokenLogprob(token='.', bytes=[46], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' **', bytes=[32, 42, 42], logprob=-0.018017534166574478, top_logprobs=[]), ChatCompletionTokenLogprob(token='Design', bytes=[68, 101, 115, 105, 103, 110], logprob=-0.25278693437576294, top_logprobs=[]), ChatCompletionTokenLogprob(token=' and', bytes=[32, 97, 110, 100], logprob=-0.33452877402305603, top_logprobs=[]), ChatCompletionTokenLogprob(token=' Brand', bytes=[32, 66, 114, 97, 110, 100], logprob=-0.514303982257843, top_logprobs=[]), ChatCompletionTokenLogprob(token='**', bytes=[42, 42], logprob=-0.18277929723262787, top_logprobs=[]), ChatCompletionTokenLogprob(token=':', bytes=[58], logprob=-0.0037415295373648405, top_logprobs=[]), ChatCompletionTokenLogprob(token=' The', bytes=[32, 84, 104, 101], logprob=-0.18502825498580933, top_logprobs=[]), ChatCompletionTokenLogprob(token=' lamp', bytes=[32, 108, 97, 109, 112], logprob=-0.10684743523597717, top_logprobs=[]), ChatCompletionTokenLogprob(token=' is', bytes=[32, 105, 115], logprob=-0.001284875557757914, top_logprobs=[]), ChatCompletionTokenLogprob(token=' designed', bytes=[32, 100, 101, 115, 105, 103, 110, 101, 100], logprob=-0.003955867141485214, top_logprobs=[]), ChatCompletionTokenLogprob(token=' by', bytes=[32, 98, 121], logprob=-9.014684110297821e-06, top_logprobs=[]), ChatCompletionToke

We can see that the model has already committed to a specific price point as part of it's analysis,
and it's condfidence is very high in a particular prediction, but this is not indicative of the true confidence.

Finally, an interesting point is that if we leave the temperature at its default value of 1, the analysis would show up a lot of less condfient predictions, which is expected given the sampling performed at non-zero temperatures:

In [7]:
eli5.explain_prediction(client, prompt_cot, model='gpt-4o')

Explanation(estimator='llm_logprobs', description=None, error=None, method=None, is_regression=False, targets=[TargetExplanation(target=Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Analysis', bytes=[65, 110, 97, 108, 121, 115, 105, 115], logprob=-3.470536947250366, top_logprobs=[]), ChatCompletionTokenLogprob(token=' of', bytes=[32, 111, 102], logprob=-0.07548464089632034, top_logprobs=[]), ChatCompletionTokenLogprob(token=' Price', bytes=[32, 80, 114, 105, 99, 101], logprob=-2.8842244148254395, top_logprobs=[]), ChatCompletionTokenLogprob(token=':\n\n', bytes=[58, 10, 10], logprob=-0.4149337112903595, top_logprobs=[]), ChatCompletionTokenLogprob(token='The', bytes=[84, 104, 101], logprob=-0.1381458044052124, top_logprobs=[]), ChatCompletionTokenLogprob(token=' product', bytes=[32, 112, 114, 111, 100, 117, 99, 116], logprob=-0.0961286872625351, top_logprobs=[]), ChatCompletionTokenLogprob(token=' description', bytes=[32, 100, 101, 115, 99, 114, 105, 112, 116, 105, 111, 110], logprob=-0.05404112488031387, top_logprobs=[]), ChatCompletionTokenLogprob(token=' does', bytes=[32, 100, 111, 101, 115], logprob=-0.7267906665802002, top_logprobs=[]), ChatCompletionTokenLogprob(token=' not', bytes=[32, 110, 111, 116], logprob=-8.41866585687967e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token=' provide', bytes=[32, 112, 114, 111, 118, 105, 100, 101], logprob=-0.8855213522911072, top_logprobs=[]), ChatCompletionTokenLogprob(token=' an', bytes=[32, 97, 110], logprob=-1.4068679809570312, top_logprobs=[]), ChatCompletionTokenLogprob(token=' explicit', bytes=[32, 101, 120, 112, 108, 105, 99, 105, 116], logprob=-0.016328897327184677, top_logprobs=[]), ChatCompletionTokenLogprob(token=' price', bytes=[32, 112, 114, 105, 99, 101], logprob=-0.0034691598266363144, top_logprobs=[]), ChatCompletionTokenLogprob(token=' for', bytes=[32, 102, 111, 114], logprob=-0.14249683916568756, top_logprobs=[]), ChatCompletionTokenLogprob(token=' the', bytes=[32, 116, 104, 101], logprob=-0.00014883324911352247, top_logprobs=[]), ChatCompletionTokenLogprob(token=' Stay', bytes=[32, 83, 116, 97, 121], logprob=-0.20818568766117096, top_logprobs=[]), ChatCompletionTokenLogprob(token=' table', bytes=[32, 116, 97, 98, 108, 101], logprob=-0.0077700018882751465, top_logprobs=[]), ChatCompletionTokenLogprob(token=' lamp', bytes=[32, 108, 97, 109, 112], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='.', bytes=[46], logprob=-0.046941064298152924, top_logprobs=[]), ChatCompletionTokenLogprob(token=' To', bytes=[32, 84, 111], logprob=-1.4835844039916992, top_logprobs=[]), ChatCompletionTokenLogprob(token=' estimate', bytes=[32, 101, 115, 116, 105, 109, 97, 116, 101], logprob=-0.0776246190071106, top_logprobs=[]), ChatCompletionTokenLogprob(token=' the', bytes=[32, 116, 104, 101], logprob=-0.6596805453300476, top_logprobs=[]), ChatCompletionTokenLogprob(token=' price', bytes=[32, 112, 114, 105, 99, 101], logprob=-0.09459269791841507, top_logprobs=[]), ChatCompletionTokenLogprob(token=',', bytes=[44], logprob=-0.016335463151335716, top_logprobs=[]), ChatCompletionTokenLogprob(token=" it's", bytes=[32, 105, 116, 39, 115], logprob=-4.817591190338135, top_logprobs=[]), ChatCompletionTokenLogprob(token=' important', bytes=[32, 105, 109, 112, 111, 114, 116, 97, 110, 116], logprob=-0.6164944171905518, top_logprobs=[]), ChatCompletionTokenLogprob(token=' to', bytes=[32, 116, 111], logprob=-3.523948907968588e-05, top_logprobs=[]), ChatCompletionTokenLogprob(token=' consider', bytes=[32, 99, 111, 110, 115, 105, 100, 101, 114], logprob=-0.00870734453201294, top_logprobs=[]), ChatCompletionTokenLogprob(token=' factors', bytes=[32, 102, 97, 99, 116, 111, 114, 115], logprob=-1.8121271133422852, top_logprobs=[]), ChatCompletionTokenLogprob(token=' such', bytes=[32, 115, 117, 99, 104], logprob=-0.13879328966140747, top_logprobs=[]), ChatCompletionTokenLogprob(token=' as', bytes=[32, 97, 115], logprob=-9.0883463

## 3. Open Source and other models

If an API endpoint can provide ``logprobs`` in the right format, then it should work. However few APIs or libraries do provide it,
even for open source models. One library which is know to work is `mlx_lm` (Mac OS only), e.g. if you start the server like this:

    mlx_lm.server --model mlx-community/Mistral-7B-Instruct-v0.3-4bit

Then you can explain predictions with a custom client:

In [5]:
client_custom = openai.OpenAI(base_url="http://localhost:8080/v1", api_key="dummy")
eli5.explain_prediction(
    client_custom,
    prompt + ' Price should never be zero.',
    model="mlx-community/Mistral-7B-Instruct-v0.3-4bit",
)

Explanation(estimator='llm_logprobs', description=None, error=None, method=None, is_regression=False, targets=[TargetExplanation(target=Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='{', bytes=[123], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='\n', bytes=[10], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' ', bytes=[32], logprob=-0.03125, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='material', bytes=[109, 97, 116, 101, 114, 105, 97, 108], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='s', bytes=[115], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='":', bytes=[34, 58], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' ["', bytes=[32, 91, 34], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='sil', bytes=[115, 105, 108], logprob=-0.078125, top_logprobs=[]), ChatCompletionTokenLogprob(token='ky', bytes=[107, 121], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' mat', bytes=[32, 109, 97, 116], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='te', bytes=[116, 101], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' grey', bytes=[32, 103, 114, 101, 121], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' metal', bytes=[32, 109, 101, 116, 97, 108], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='"],', bytes=[34, 93, 44], logprob=-0.015625, top_logprobs=[]), ChatCompletionTokenLogprob(token='\n', bytes=[10], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' ', bytes=[32], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='type', bytes=[116, 121, 112, 101], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='":', bytes=[34, 58], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=-0.484375, top_logprobs=[]), ChatCompletionTokenLogprob(token='Not', bytes=[78, 111, 116], logprob=-0.859375, top_logprobs=[]), ChatCompletionTokenLogprob(token=' specified', bytes=[32, 115, 112, 101, 99, 105, 102, 105, 101, 100], logprob=-0.046875, top_logprobs=[]), ChatCompletionTokenLogprob(token=' in', bytes=[32, 105, 110], logprob=-0.6875, top_logprobs=[]), ChatCompletionTokenLogprob(token=' the', bytes=[32, 116, 104, 101], logprob=-0.125, top_logprobs=[]), ChatCompletionTokenLogprob(token=' description', bytes=[32, 100, 101, 115, 99, 114, 105, 112, 116, 105, 111, 110], logprob=-0.265625, top_logprobs=[]), ChatCompletionTokenLogprob(token='",', bytes=[34, 44], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='\n', bytes=[10], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' ', bytes=[32], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='color', bytes=[99, 111, 108, 111, 114], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='":', bytes=[34, 58], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' "', bytes=[32, 34], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='Not', bytes=[78, 111, 116], logprob=-0.3125, top_logprobs=[]), ChatCompletionTokenLogprob(token=' specified', bytes=[32, 115, 112, 101, 99, 105, 102, 105, 101, 100], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' in', bytes=[32, 105, 110], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' the', bytes=[32, 116, 104, 101], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' description', bytes=[32, 100, 101, 115, 99, 114, 105, 112, 116, 105, 111, 110], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='",', bytes=[34, 44], logprob=0.0, top_logprobs=[]), ChatCo